In [1]:
# import
import numpy as np
import pandas as pd
import math
import os
import random
import scipy.stats as stats
from scipy.stats import ks_2samp
from scipy.stats import entropy
import copy
import pickle
import itertools
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score
from datetime import timedelta, datetime
np.set_printoptions(suppress=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                # show multi variables without print

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
from scipy.stats import gamma

In [8]:
def get_gamma_pdfs(x, params):
    # x: a list of samples, for example, x=[1,2,3,4]
    # params: mixing, shape and scale of gamma mix, for example, [[1,0.1,0.2]]
    y = np.zeros(len(x))    
    for t in np.arange(len(params)):
        mixing, alpha, beta = params[t]
        temp_y = mixing * stats.gamma.pdf(x, alpha, loc=0, scale=beta)
        y = y + temp_y
    return y

def find_index_windows(series):
    if not series:
        return {}

    index_windows = {}
    start = 0

    while start < len(series):
        current_value = series[start]
        end = start
        
        while end < len(series) and series[end] == current_value:
            end += 1
        
        if current_value not in index_windows:
            index_windows[current_value] = []
        
        index_windows[current_value].append((start + 1, end))  # Use 1-based indexing
        start = end

    return index_windows

def plot_pdf_sf_cp(x, y, ID, K, tag, Path_data, index_windows, path_save):
    nbins = 50

    lightBlue = (0.6, 0.8, 1.0)
    darkBlue = (0.2, 0.4, 0.8)
    lightOrange = (1.0, 0.8, 0.6)
    darkOrange = (0.9, 0.5, 0.2)
    greenLine = (0.2, 0.7, 0.3)
    redLine = (0.8, 0.3, 0.3)
    purpleLine = (0.5, 0.2, 0.7)

    data = pd.read_csv(Path_data + str(ID) + '.csv').values

    tags = data[:, 4]
    dataCRC_W1_C = data[(tags == 1) | (tags == 2), :]
    dataCRC_W1_RC = data[(tags == 3) | (tags == 4) | (tags == 5) | (tags == 6), :]

    DurCRC_W1_C = dataCRC_W1_C[:, 1] + dataCRC_W1_C[:, 2]
    DurCRC_W1_RC = dataCRC_W1_RC[:, 1] + dataCRC_W1_RC[:, 2]

    DurCRC_W1_C = DurCRC_W1_C.T
    DurCRC_W1_RC = DurCRC_W1_RC.T

    # ================= plot
    fig, ax = plt.subplots(figsize=(6, 3.6))
    ploot = plt.xlim([0,12000])
    if tag == 'SF' or tag == 'CP':
        ploot = plt.ylim([0,1])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
    if tag == 'PDF':
        minX = 0
        maxX = 12000
        minY = 0
        maxY = 0.7 * 0.001
        stride = 300
        binEdges = np.arange(0, maxX + stride, stride)
        Nedges = binEdges.size
    
        # =============== figure 1 for W1
        countsA = np.histogram(DurCRC_W1_C, bins=binEdges)[0]
        countsB = np.histogram(DurCRC_W1_RC, bins=binEdges)[0]
        densitiesA = countsA / (np.sum(countsA + countsB) * stride)
        densitiesB = countsB / (np.sum(countsA + countsB) * stride)
        newX = binEdges[:-1] + stride / 2
        newY = np.vstack((densitiesA, densitiesB)).T

        ploot = ax.bar(newX, newY[:, 0], width=stride, color=lightBlue, edgecolor=darkBlue, label='Complete')
        ploot = ax.bar(newX, newY[:, 1], width=stride, color=lightOrange, edgecolor=darkOrange, bottom=newY[:, 0], label='Right-censored')

    for k in np.arange(K):       
        ploot = plt.plot(x, y[:,k].reshape(-1,1), color=colour_names[k], linestyle='-', label='gamma' + str(k+1))
    if tag == 'PDF' or tag == 'SF':
        ploot = plt.plot(x, y[:,K], label='mix', linestyle='--', color='g')
        
    if tag == 'CP':
        highlight_color = ['lightblue', 'lightyellow', 'lightgreen', (1.000, 0.667, 0.667)]
        for key, value in index_windows.items():
            first = True
            for window in value:
#                 print(key, window, highlight_color[key])
                t_window = [t[window[0]-1], t[window[1]-1]]
                if first:
                    first = False
                    ax.axvspan(t_window[0], t_window[1], color=highlight_color[key], alpha=0.5, label='cluster'+str(key+1))
                else:
                    ax.axvspan(t_window[0], t_window[1], color=highlight_color[key], alpha=0.5)
    
    if tag == 'CP':
        ploot = plt.legend(prop=font_legend, bbox_to_anchor=(1.3, 0.6))
    else:
        ploot = plt.legend(prop=font_legend) 

    if tag == 'PDF':
        ploot = ax.set_xlim([minX, maxX])
        ploot = ax.set_ylim([minY, maxY])

    if tag == 'PDF':
        ploot = ax.set_ylabel("PDF", fontdict=font)
    if tag == 'SF':
        ploot = ax.set_ylabel("SF", fontdict=font)
    if tag == 'CP':
        ploot = ax.set_ylabel("CPF", fontdict=font)

    my_x_ticks = ax.get_xticks()
    my_y_ticks = ax.get_yticks()
    my_x_ticks = np.round(my_x_ticks, decimals=3).astype(int)
    my_y_ticks = np.round(my_y_ticks, decimals=4)
    ploot = ax.set_xticks(my_x_ticks)
    ploot = ax.set_xticklabels(my_x_ticks, fontdict=font)
    ploot = ax.set_yticks(my_y_ticks)
    ploot = ax.set_yticklabels(my_y_ticks, fontdict=font)
    ploot = ax.set_xlabel("hours", fontdict=font)
    
    ploot = plt.savefig(path_save, dpi = DPI, bbox_inches = 'tight')  
    ploot = plt.close()

In [3]:
Path_data = '../Sec3 Model fitting/data/window3years/'
data = pd.read_csv(Path_data + '1.csv')
data

,ID,y1,y2,y3,tag
0,0,365.450000,8450.733333,0.000000,1
1,23080,327.883333,4629.266667,0.000000,1
2,34618,279.916667,1635.366667,0.000000,1
3,38464,327.833333,4314.383333,0.000000,1
4,46156,158.400000,2433.866667,0.000000,1
...,...,...,...,...,...
48598,5783,0.000000,0.000000,128.433333,8
48599,95280,0.000000,0.000000,7.800000,8
48600,15105,0.000000,0.000000,38.783333,8
48601,15252,0.000000,0.000000,59.333333,8


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ==================================== settings ========================
Path_data = '../Sec3 Model fitting/data/window3years2/'
Path_model = '../Sec3 Model fitting/out3Years2/long run/'
Path_pdf = 'OfflineClustering/PDF/'
Path_sf = 'OfflineClustering/SF/'
Path_cp = 'OfflineClustering/CP/'

font = {'weight':"bold",'size':12}
fontsize = 12
font_legend = {'size':10}
DPI = 500
nbins = 50
colour_names = ['b', 'r', 'c', 'k', 'm']

IDs = ['2013', '2014', '2015']
t_max = 12000
t = np.arange(0, t_max, 1)

for sliding_window, ID in zip(np.arange(1, len(IDs)+1), IDs):
    print(sliding_window)
    
    W_file = f"{Path_data}{ID}.csv"
    data = pd.read_csv(W_file).to_numpy()  # data to be fitted
    n = data.shape[0]

    # ================== CRC
    paramCRC_W1 = pd.read_csv(Path_model + str(sliding_window) + "/W1.csv").to_numpy()
    means = paramCRC_W1[:, 1] * paramCRC_W1[:, 2]  # calculate means
    paramCRC_W1 = np.concatenate((paramCRC_W1, means.reshape(-1,1)), axis=1)

    cols = ['weight', 'shape', 'scale','mean']  
    df_W1 = pd.DataFrame(paramCRC_W1, columns=cols)
    parameters1 = df_W1.sort_values(by=['mean'], ascending=True).values
    K1 = len(parameters1)

    # ======== Calculate survival values for individual gamma component
    ft1, St1 = [], []   # values of the density function of individual gamma component under time points 
    for k in np.arange(K1):
        St1.append(gamma.sf(t, parameters1[k][1], loc=0, scale=parameters1[k][2])) # the survival value 
        ft1.append( gamma.pdf(t, parameters1[k][1], loc=0, scale=parameters1[k][2]) )    
    St1 = np.array(St1)
    ft1 = np.array(ft1)   

    mixSt1 = np.sum(St1 * parameters1[:,0].reshape(-1,1), axis=0)   # survival values of the mixture model
    mixft1 = np.sum(ft1 * parameters1[:,0].reshape(-1,1), axis=0)   # density values of the mixture model
    out_St1 = np.concatenate( (St1, mixSt1.reshape(1,-1)), axis=0 ).T   # survival values of single and mix
    out_ft1 = np.concatenate( (ft1, mixft1.reshape(1,-1)), axis=0 ).T   # density values of single and mix
    
    # ======== Calculate the conditional probability
    Cprob1 = []   # values of conditional probability
    for k in np.arange(K1):
        con_prob = parameters1[k][0] * ft1[k] / mixft1
        Cprob1.append(con_prob)
    Cprob1 = np.array(Cprob1).T
    
    # ======== Save ft, St, Cprob
    pd.DataFrame(out_St1).to_csv(Path_sf + ID + '.csv', index=False)
    pd.DataFrame(out_ft1).to_csv(Path_pdf + ID + '.csv', index=False)
    pd.DataFrame(Cprob1).to_csv(Path_cp + ID + '.csv', index=False)
        
K = [2,3,3]
for ID, k in zip(IDs, K):
    # ================ Plot Density
    St = pd.read_csv(Path_sf + ID + '.csv').values
    ft = pd.read_csv(Path_pdf + ID + '.csv').values
    Cprob = pd.read_csv(Path_cp + ID + '.csv').values
    
    tags = np.argmax( Cprob, axis=1 )
    index_windows = find_index_windows(tags.tolist())
    
    plot_pdf_sf_cp(t, ft, ID, k, 'PDF', Path_data, index_windows, 'OfflineClustering_pdf_' + str(ID) + '.png')
    plot_pdf_sf_cp(t, St, ID, k, 'SF', Path_data, index_windows, 'OfflineClustering_sf_' + str(ID) + '.png')
    plot_pdf_sf_cp(t, Cprob, ID, k, 'CP', Path_data, index_windows, 'OfflineClustering_cp_' + str(ID) + '.png')   

1
2
3
